# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import Workspace, Experiment
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'your experiment name here'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = <your policy here>

#TODO: Create the different params that you will be using during training
param_sampling = <your params here>

#TODO: Create your estimator and hyperdrive config
estimator = <your estimator here>

hyperdrive_run_config = <your config here?

In [ ]:
# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.5, 0.99),
    "--max_iter": choice(1, 2, 4, 8, 16, 32, 64, 128)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 2, slack_factor = 0.1)

# Create a SKLearn estimator for use with train.py
est = SKLearn(
        source_directory=project_folder,
        entry_script=training_script_name,
        script_params={'--C':1.0, '--max_iter':2},
        compute_target=cpu_cluster
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                         hyperparameter_sampling=ps,
                                         policy=policy,
                                         primary_metric_name=primary_metric_name,
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=4,
                                         max_concurrent_runs=4)

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

best_run = hyperdrive_run.get_best_run_by_primary_metric()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print(parameter_values)

model_parameters = dict(zip(parameter_values[::2], parameter_values[1::2]))
model_parameters['--save'] = True

from azureml.train.estimator import Estimator
model_est = Estimator(source_directory='.',
                      entry_script=training_script_name,
                      script_params=model_parameters,
                      compute_target=cpu_cluster,
                      conda_packages=['pandas==0.23.4',
                                      'scikit-learn==0.21.3',
                                      'lightgbm==2.2.1',
                                      'skits=0.1.2'])

model_run = exp.submit(model_est)
model_run.wait_for_completion(show_output=True)

In [ ]:
#TODO: Submit your experiment

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service